## plottig ideas, examples

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import vmlab
from vmlab.models import vmango

In [ ]:
tree = pd.read_csv('../../share/trees/tree_B10_cycle_030405.csv')

In [ ]:
setup = vmlab.create_setup(
    model=vmango,
    tree=tree,
    start_date='2003-06-01',
    end_date='2005-06-01',
    setup_toml='../../share/setup/vmango.toml',
    current_cycle=3,
    input_vars={
        'geometry__interpretation_freq': 0
    },
    output_vars={
        'fruit_quality': {
            'FM_fruit': 'day'
        },
        'carbon_balance': {
            'DM_fruit': 'day'
        }
    }
)

In [ ]:
ds_out = vmlab.run(setup, vmango)

In [ ]:
# select variables, the date range that is important, rename variables
ds = ds_out[['fruit_quality__FM_fruit', 'carbon_balance__DM_fruit']].sel({
        'day': slice('2004-11-01', '2005-03-01')
    }).where(ds_out.carbon_balance__DM_fruit > 0, drop=True).rename_vars({
        'fruit_quality__FM_fruit': 'FM',
        'carbon_balance__DM_fruit': 'DM',
    })
ds

In [ ]:
# move variables and their values into one column so it is useable for seaborn facegrids
df = pd.melt(ds.to_dataframe().reset_index(), id_vars=['GU', 'day'], value_vars=['FM', 'DM'], var_name='vars', value_name='gram')
df

In [ ]:
grid = sns.FacetGrid(df, col='GU', hue='vars', col_wrap=6)
grid.map(sns.lineplot, 'day', 'gram')
for ax in grid.axes.ravel():
    ax.legend()